In [308]:
import gym
import gym_gigticket
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


env = gym.make('gigticket-v0')

# Experience Replay

In [309]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

# Network Architecture

In [310]:
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(3, 128, bias = False),
            #nn.BatchNorm1d(128),
            nn.ReLU(True), 
            nn.Linear(128, 256, bias = False),
            #nn.BatchNorm1d(256), 
            nn.ReLU(True), 
            nn.Linear(256, 128, bias = False),
            #nn.BatchNorm1d(128), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.Linear(128, 64, bias = False),
            #nn.BatchNorm1d(64), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.Linear(64, 3, bias = False),
            nn.Softmax(dim=1), # We apply another ReLU.
        )

    def forward(self, input):
        output = self.main(input) 
        return output
        
        

        ##return self.head(x.view(x.size(0), -1))

# Action Selection

In [311]:
# Get number of actions from gym action space
n_actions = env.action_space.n

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            state_tensor = torch.tensor([state]).to(dtype= torch.float)
            return policy_net(state_tensor).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], dtype=torch.long)

# Hyperparameters and nets

In [312]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

memory = ReplayMemory(10000)

episode_durations = []

policy_net = DQN()
# Need to make sure the data and model parameters have the same data type.
policy_net = policy_net.float()
target_net = DQN()
target_net = target_net.float()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

# create optimizer.
optimizer = optim.RMSprop(policy_net.parameters())

steps_done = 0

# Model Optimizer

In [313]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None]).to(dtype= torch.float)
    
    state_batch = torch.cat(batch.state).to(dtype= torch.float)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    #print('LOSS {}'.format(loss))
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
    return loss

# Main Training Loop

In [ ]:
'''
Had some real issues with mutable objects in Python here and I'm not sure they've been fixed particularly well.

Because the gym class saves the state as a list, a list is mutable, so when I try and record the curretn state and the next state
the current state gets updated to the next state 'in the backgroud' because they're just looking to the same place in memory.
This obvisouly messes up this approach and i bodge it by using old_state = state[:] which is a bodge way to clone. Not sure how to do
this properly. 
'''

num_episodes = 5000
for i_episode in range(num_episodes):
    # Initialize the environment and state
    state = env.reset()[:]
    
    for t in count():
        # Select and perform an action
        action = select_action(state)
        next_state, reward, done, _ = env.step(action.item())
 
        # Observe new state
        if done:
            next_state = None
            next_state_save = None
        else:
            next_state_save = next_state[:]
            next_state_save = torch.tensor([next_state_save])

        # Store the transition in memory
        
        reward = torch.tensor([reward])
        state_save = state[:]
        state_save = torch.tensor([state_save])
        
        memory.push(state_save, action, next_state_save, reward)

        # Perform one step of the optimization (on the target network)
        loss = optimize_model()
        
        if done:
            episode_durations.append(t + 1)
            #plot_durations()
            break
        else:
            #Move to the next state
            state = next_state[:]
            
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        
        target_net.load_state_dict(policy_net.state_dict())
    if i_episode % 1000 == 0 :
        print('loss {} at epiode {}'.format(loss,i_episode))

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

loss None at epiode 0


# Currently unused:

In [ ]:
def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())